In [1]:
import pandas as pd
from os import chdir
import subprocess
import glob

# local import (Hirotaka's myfunctions.py script)
import myfunctions as myfunc

In [2]:
!mkdir temp
chdir('temp')

mkdir: cannot create directory 'temp': File exists


In [3]:
# Get the id file, covariate file (all European)
wgsfolder = '/data/CARD/PD/GENOMES/august19/genotypes'
idfile = '_keep.id'
demog = pd.read_csv('/data/LNG/iwakih2/ampPD/data1_out/PP_PD_BF_HB_LC_demog.csv')
euro = pd.read_csv(f'{wgsfolder}/PCA_filtered_europeans.txt', sep=' ',
                  header=None, names=['FID', 'IID'])
# df = pd.merge(demog, euro, left_on='WGSID', right_on='IID')
df = demog
df['FID'] = df.WGSID
df['IID'] = df.WGSID
df.RECRUIT=['CTR' if i=='HC' else i for i in df.RECRUIT]
df = df[df.RECRUIT.isin(['CTR', 'PD'])]
df = df[df.RECRUIT==df.LASTDIAG]
df['pheno'] = [1 if i=='PD' else 0 for i in df.RECRUIT]
df['AAD'] = df.AADi.where(df.pheno==0, other=df.AAOi)
df['AGE'] = df.AGEatBL.where(df.pheno==0, other=df.AAD)

df[['FID', 'IID', 'pheno']].to_csv(idfile, index=False, sep='\t', header=None)
df[['FID', 'IID', 'AGE', 'FEMALE']].to_csv('_qcovar.txt', index=False, sep='\t', header=None)

In [ ]:
# for chrnum in range(1,23):
#     pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
#     out = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#     plinkcmd = f'\
#  plink2 --pfile {pfile}\
#  --make-bed\
#  --out {out}'
#    myfunc.shell_do(plinkcmd)

In [ ]:
# with open('_mergelist.txt', 'w') as f:
#     for chrnum in range(1,23):
#         bfile = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#         f.write(f'{bfile}\n')
    

In [ ]:
# mergecmd = f'\
# plink --merge-list _mergelist.txt\
#  --make-bed\
#  --out {wgsfolder}/merge/pd.june2019.sqc'
# with open('_merge_batch_script.sh', 'w') as f:
#     f.write('#!/bin/bash\n')
#     f.write('module load plink\n')
#     f.write(f'{mergecmd}')

In [ ]:
# !sbatch _merge_batch_script.sh --cpus-per-task=128 --mem=100g

In [7]:
with open('_pca.swarm', 'w') as f:
    for chrnum in range(1,2):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        out_pcs = f'chr{chrnum}.pca'
        plink_pca_cmd = f'\
plink2 --pfile {pfile}\
 --pca 50\
 --out {out_pcs}\n'
        
        f.write(plink_pca_cmd)
        print(plink_pca_cmd)

plink2 --pfile /data/CARD/PD/GENOMES/august19/genotypes/pd.june2019.chr1.sqc --pca 50 --out chr1.pca



In [8]:
!swarm -f _pca.swarm -g 64 --time=10:00:00 -t 20 --logdir swarm --module plink/2.0_alpha_1_final --partition=norm

60297240


In [ ]:
with open('_script1.sh', 'w') as f:
    for chrnum in range(1,):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        plinkcmd = f"\
plink2 --pfile {pfile}\
 --keep {idfile} \
 --freq \
 --out {freqfile}\n"
        f.write(plinkcmd)
#         myfunc.shell_do(plinkcmd)
print(f'{plinkcmd}')

In [4]:
from dask.distributed import Client, progress
# client should match the qued number of threads
client = Client(processes=False, threads_per_worker=2,
                n_workers=2, memory_limit='30GB')
# import dask
import dask.dataframe as dd
def MAFbin(x):
    if x>0:
        if x<0.001:
            return(1)
        elif x<0.01:
            return(2)
        elif x<0.1:
            return(3)
        elif x<0.2:
            return(4)
        elif x<0.3:
            return(5)
        elif x<0.4:
            return(6)
        else:
            return(7)
    else:
        return(0)

/usr/local/Anaconda/envs/py3.7/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [13]:
client.close()

In [ ]:
for chrnum in range(1,23):
    freqfile = f'freq.{chrnum}'

    df = dd.read_csv(f'{freqfile}.afreq', sep='\t')
    df['MAF'] = df.ALT_FREQS.where(cond=df.ALT_FREQS<=0.5, other=1-df.ALT_FREQS)
    df['Group'] = df.MAF.apply(MAFbin, meta=('MAF', 'float64'))
    for i in range(1,8):
        df.loc[df.Group==i, ['ID']].to_csv(
            f'{freqfile}.mafbin{i}', index=False, single_file=True)
    print(f'chr{chrnum} finished')

In [ ]:
with open('_script2.swarm', 'w') as f:
    for chrnum in range(1,23):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        for Indx in range(1,8):
            mafbinIndx = f'mafbin{Indx}'
            mafbinfile =  f'{freqfile}.{mafbinIndx}'

            plinkcmd = f"\
plink2 --pfile {pfile}\
 --extract {mafbinfile}\
 --keep {idfile}\
 --make-bed\
 --out {mafbinfile}"
            gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin'

            f.write(f'{plinkcmd} && {gctacmd1}\n')

In [ ]:
# !swarm -f _script2.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA --partition=norm --devel

In [ ]:
# need to rerun for jobs that were killed
outputs = glob.glob('freq.*.mafbin*.ldbin.score.ld')
# len(glob.glob('*.ldbin.log'))
expected_out = ['freq.' + str(i) + '.mafbin' + str(j) + '.ldbin.score.ld' for i in range(1,23) for j in range(1,8)]
missing = [bed.replace('.ldbin.score.ld', '') for bed in list(set(expected_out) - set(outputs))]

with open('_rerunscript.swarm', 'w') as f:
    for mafbinfile in missing:
        gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin\
 --thread-num 10'

        f.write(f'{gctacmd1}\n')

In [ ]:
# !swarm -f _rerunscript.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA --partition=norm --devel

In [ ]:
for chrnum in range(1,23):
    pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
    freqfile = f'freq.{chrnum}'
    for Indx in range(1,8):
        mafbinIndx = f'mafbin{Indx}'
        mafbinfile =  f'{freqfile}.{mafbinIndx}'

        
        

        # Note this is the ldscore within the maf bin
        df = dd.read_csv(f'{mafbinfile}.ldbin.score.ld', sep=' ')
        ldscvec = df.ldscore_SNP.compute()
        ldscThres = ldscvec.median()
        df['Group'] = 1
        df['Group'] = df.Group.where(cond=df.ldscore_SNP<ldscThres, other=2)


        # Create ldbin and GRMs within the ldbin
        for i in range(1, 3):
            ldbinIndx = f'{mafbinfile}.ldbin{i}'
            df.loc[df.Group==i, ['SNP']].to_csv(
                f'{ldbinIndx}', index=False, single_file=True)
            gctacmd2 = f'\
            gcta64 --bfile {mafbinfile}\
 --extract {ldbinIndx}\
 --make-grm\
 --out {ldbinIndx}'
            myfunc.shell_do(gctacmd2)
            print(gctacmd2)
            with open('multi_GRMs.txt', 'a') as f:
                f.write(f'{ldbinIndx}\n')
        # mafbin information save to mafbinInfo.csv
        with open('mafbinInfo.txt', 'a') as f:
            s = f'{chrnum}\t{mafbinIndx}\t{len(ldscvec)}\t{ldscvec.mean()}\t{ldscvec.median()}\t{ldscvec.std()}'
            f.write(f'{s}\n')

In [40]:
# create covar file per chrom and impute missing age (may want to consider dropping age column in this case (missing for more than half))
# impute "FEMALE" with median
for chrnum in range(1,2):
    pcs_df = pd.read_csv(f'chr{chrnum}.pca.eigenvec', sep='\t')
    pcs_df.rename(columns={'#FID':'FID'}, inplace=True)
    covar_df = df[['FID', 'IID', 'AGE', 'FEMALE']].merge(pcs_df, on=['FID','IID'])
    covar_df.AGE.fillna(covar_df.AGE.mean(), inplace=True)
    covar_df.FEMALE.fillna(covar_df.FEMALE.median(),inplace=True)
#     print(f'dropping {covar_df[covar_df.AGE.isnull()].shape[0]} rows for missing age')
#     covar_df.dropna(how='any', inplace=True)
    covar_df.to_csv(f'chr{chrnum}.covar', index=False, sep=' ', header=None)
    print(f'wrote: chr{chrnum}.covar with {covar_df.shape[0]} samples')

wrote: chr1.covar with 3387 samples


In [48]:
pcs_df.shape

(9300, 52)

In [11]:
#  --pheno {idfile}\

gctacmd3 = f'\
gcta --reml\
 --mgrm multi_GRMs.txt\
 --pheno _fake_pheno.txt\
 --qcovar chr1.pca.eigenvec\
 --reml-no-lrt\
 --thread-num 8\
 --out test'
myfunc.shell_do(gctacmd3)

# with open('_reml_script.sh', 'w') as f:
#     f.write('#!/bin/bash\n')
#     f.write(f'{gctacmd3}\n')

Executing: gcta --reml --mgrm multi_GRMs.txt --pheno _fake_pheno.txt --qcovar chr1.pca.eigenvec --reml-no-lrt --thread-num 8 --out test


In [50]:
!sbatch --mem=100g --cpus-per-task=64 _reml_script.sh

60400321


In [69]:
# df[['FID', 'IID', 'pheno']]
df.pheno.value_counts()

1    3949
0    3239
Name: pheno, dtype: int64

In [57]:
euro[['FID']].isnull().sum()

FID    0
dtype: int64

In [8]:
import numpy as np
# making fake pheno data for testing
euro['pheno'] = np.random.choice([0, 1], size=(euro.shape[0],), p=[2./3, 1./3])

array([1, 1, 1, ..., 1, 0, 1])

In [10]:
euro.to_csv('_fake_pheno.txt', sep='\t', header=None)